
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [9]:
%%time
geo['CP Ville'] = geo['Code Postal'] + ' ' + geo['Commune']

Wall time: 30 ms


In [10]:
%%time
geo['CP Ville'] = geo['Code Postal'].combine(geo['Commune'], lambda x, y: f'{x} {y}')

Wall time: 1.26 s


In [11]:
# fonction recherche de ville
def ville(lat, long):
    dist = np.sqrt(((geo['Latitude']-lat)**2 + (geo['Longitude']-long)**2))
    return geo.loc[dist.idxmin(), 'CP Ville']

In [17]:
# fonction recherche de ville
def ville(lat, long):
    dist2 = ((geo['Latitude']-lat) ** 2 + (geo['Longitude']-long) ** 2)
    return geo.loc[dist2.idxmin(), 'CP Ville']

In [18]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'71330 BOSJEAN'

In [14]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [15]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

In [16]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 22, 50), dms2dec(0, 51, 1))

'61110 CONDE-SUR-HUISNE'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [21]:
import json

s_json = geo['geo_shape'].apply(json.loads)
s_json

Code INSEE
01001    {'type': 'Polygon', 'coordinates': [[[4.926273...
01002    {'type': 'Polygon', 'coordinates': [[[5.430089...
01004    {'type': 'Polygon', 'coordinates': [[[5.386190...
01005    {'type': 'Polygon', 'coordinates': [[[4.895580...
01006    {'type': 'Polygon', 'coordinates': [[[5.614854...
01007    {'type': 'Polygon', 'coordinates': [[[5.413533...
01008    {'type': 'Polygon', 'coordinates': [[[5.321986...
01009    {'type': 'Polygon', 'coordinates': [[[5.656393...
                               ...                        
97610    {'type': 'Polygon', 'coordinates': [[[45.23692...
97611    {'type': 'Polygon', 'coordinates': [[[45.19781...
97612    {'type': 'MultiPolygon', 'coordinates': [[[[45...
97613    {'type': 'Polygon', 'coordinates': [[[45.10168...
97614    {'type': 'Polygon', 'coordinates': [[[45.15401...
97615    {'type': 'Polygon', 'coordinates': [[[45.29645...
97616    {'type': 'Polygon', 'coordinates': [[[45.13226...
97617    {'type': 'Polygon', 'coordinates': [

In [24]:
s_json.apply(lambda x: x['type']).value_counts()

Polygon         36670
MultiPolygon       72
Name: geo_shape, dtype: int64

In [28]:
s_json.apply(lambda x: x['coordinates']).apply(len).value_counts()

1    36660
2       80
4        1
3        1
Name: geo_shape, dtype: int64

In [29]:
geo.loc[s_json.apply(lambda x: x['coordinates']).apply(len)==4, 'CP Ville']

Code INSEE
83069    83400 HYERES
Name: CP Ville, dtype: object

In [30]:
geo.loc[s_json.apply(lambda x: x['coordinates']).apply(len)==3, 'CP Ville']

Code INSEE
27701    27100 VAL-DE-REUIL
Name: CP Ville, dtype: object

In [36]:
geo.loc[s_json.apply(lambda x: x['type']=='Polygon')
        & (s_json.apply(lambda x: x['coordinates']).apply(len)==2), 'CP Ville']

Code INSEE
02232               02600 COYOLLES
05052               05600 EYGLIERS
09042    09240 LA BASTIDE-DE-SEROU
22209              22650 PLOUBALAY
2A272                20100 SARTENE
2B049              20214 CALENZANA
48080               48300 LANGOGNE
67486              67920 SUNDHOUSE
68078              68420 EGUISHEIM
71028            71270 BEAUVERNOIS
Name: CP Ville, dtype: object

In [37]:
from shapely.geometry import Polygon

def test_enclave(coordinates):
    if len(coordinates) != 2:
        return False
    return Polygon(coordinates[0]).contains(Polygon(coordinates[1]))

s_json.loc[s_json.apply(lambda x: x['type']=='Polygon')
           & (s_json.apply(lambda x: x['coordinates']).apply(len)==2)].apply(lambda x: test_enclave(x['coordinates'])).all()

True